In [9]:
import re

def read_input(filename):
    with open(filename, 'r') as file:
        alpha = file.readline().strip()
        num_clauses = int(file.readline().strip())
        clauses = [re.split(r'\s+OR\s+', clause) for clause in file.readlines()]
        clauses = [list(literal.strip() for literal in clause) for clause in clauses]
    print(f"Perform read_input {filename}")
    print('alpha: ', alpha)
    print('clauses: ', clauses)
    print("--------------------")
    return alpha, clauses

def write_output(filename, results):
    print(f"Perform write_output {filename} with the results: {results}")
    print("--------------------")
    with open(filename, 'w') as file:
        final_result = ""
        if results:
            for loop_result in results:
                file.write(f"{len(loop_result)}\n")
                formatted_clause = ' OR '.join(loop_result)
                file.write(f"{formatted_clause}\n")
                final_result = "YES" if len(loop_result) == 0 else "NO"
                file.write("\n")
        else:
            final_result = "NO"

        file.write(final_result)

def pl_resolution(alpha, kb_clauses, output_filename):
    results = []
    neg_alpha = f"-{alpha}"
    # check if neg_alpha contain '--' then remove it and leave only one '-' or none
    while('--' in neg_alpha):
        neg_alpha = neg_alpha.replace('--', '')

    kb_clauses.append([neg_alpha])

    # remove duplicate clauses in kb_clauses in list of list
    kb_clauses = [list(x) for x in set(tuple(x) for x in kb_clauses)]

    for clause in kb_clauses:
        clause.sort()
    kb_clauses.sort()

    print(f"Perform pl_resolution with alpha: {alpha} and kb_clauses: {kb_clauses}")
    print("--------------------")

    # Initial loop
    current_loop = kb_clauses.copy()
    unique_clauses = current_loop.copy()

    while True:
        new_clauses = set()
        for ci, clause_i in enumerate(current_loop):
            for cj in range(ci + 1, len(current_loop)):
                clause_i = set(clause_i)
                clause_j = set(list(current_loop)[cj])
                resolvents = resolve(clause_i, clause_j)
                new_clauses.update(resolvents)

        if new_clauses:
            new_clauses = {frozenset(remove_opposite_literals(clause)) for clause in new_clauses}
            new_clauses -= {clause for clause in new_clauses if clause == frozenset()}
            new_clauses -= {frozenset(clause) for clause in new_clauses if clause in unique_clauses}

        if frozenset() in new_clauses:
            break  # Empty clause found, KB entails alpha

        if new_clauses.issubset(frozenset(clause) for clause in current_loop):
            break  # No new clauses can be generated

        # Only update if new clauses are encountered
        unique_new_clauses = new_clauses - set(frozenset(clause) for clause in unique_clauses)
        unique_new_clauses = list(list(clause for clause in clauses) for clauses in unique_new_clauses)
        if unique_new_clauses:
            unique_clauses.extend(unique_new_clauses)
            current_loop.extend(unique_new_clauses)
            results.extend(current_loop.copy())

        
    print(f"Perform pl_resolution with alpha: {alpha} and kb_clauses: {kb_clauses}")
    print('results: ', results)
    print("--------------------")
    write_output(output_filename, results)
    return results

def resolve(clause_i, clause_j):
    resolvents = set()
    for literal in clause_i:
        neg_literal = f"-{literal}"
        if neg_literal in clause_j:
            resolvent = (clause_i - {literal}) | (clause_j - {neg_literal})
            resolvent -= {""}  
            resolvents.add(frozenset(resolvent))
    print(f"Perform resolve with clause_i: {clause_i} and clause_j: {clause_j}")
    print('resolvents: ', resolvents)
    print("--------------------")
    return resolvents

def remove_opposite_literals(clause):
    # Separate positive and negative literals
    positive_literals = {literal for literal in clause if not literal.startswith('-')}
    negative_literals = {literal[1:] for literal in clause if literal.startswith('-')}
    
    # Remove opposite literals from the clause
    clause_without_opposites = positive_literals - negative_literals
    
    # Return clause after removing opposite pairs
    return clause_without_opposites

def main(input_filename, output_filename):
    alpha, kb_clauses = read_input(input_filename)
    pl_resolution(alpha, kb_clauses, output_filename)



In [2]:
main("input1.txt", "output1.txt")

Perform read_input input1.txt
alpha:  A
clauses:  [['-A', 'B', 'D'], ['B', '-C', '-D'], ['-B'], ['B', 'C']]
--------------------
Perform pl_resolution with alpha: A and kb_clauses: [['-A'], ['-A', 'B', 'D'], ['-B'], ['-C', '-D', 'B'], ['B', 'C']]
--------------------
Perform resolve with clause_i: {'-A'} and clause_j: {'D', 'B', '-A'}
resolvents:  set()
--------------------
Perform resolve with clause_i: {'-A'} and clause_j: {'-B'}
resolvents:  set()
--------------------
Perform resolve with clause_i: {'-A'} and clause_j: {'-C', 'B', '-D'}
resolvents:  set()
--------------------
Perform resolve with clause_i: {'-A'} and clause_j: {'B', 'C'}
resolvents:  set()
--------------------
Perform resolve with clause_i: {'D', 'B', '-A'} and clause_j: {'-B'}
resolvents:  {frozenset({'D', '-A'})}
--------------------
Perform resolve with clause_i: {'D', 'B', '-A'} and clause_j: {'-C', 'B', '-D'}
resolvents:  {frozenset({'-C', 'B', '-A'})}
--------------------
Perform resolve with clause_i: {'D', '

In [3]:
main("input2.txt", "output2.txt")

Perform read_input input2.txt
alpha:  L
clauses:  [['-L', 'M', '-N'], ['-M', 'N', '-L'], ['L', '-M', 'N']]
--------------------
Perform pl_resolution with alpha: L and kb_clauses: [['-L'], ['-L', '-M', 'N'], ['-L', '-N', 'M'], ['-M', 'L', 'N']]
--------------------
Perform resolve with clause_i: {'-L'} and clause_j: {'-M', 'N', '-L'}
resolvents:  set()
--------------------
Perform resolve with clause_i: {'-L'} and clause_j: {'M', '-N', '-L'}
resolvents:  set()
--------------------
Perform resolve with clause_i: {'-L'} and clause_j: {'-M', 'N', 'L'}
resolvents:  set()
--------------------
Perform resolve with clause_i: {'-M', 'N', '-L'} and clause_j: {'M', '-N', '-L'}
resolvents:  {frozenset({'-M', 'M', '-L'})}
--------------------
Perform resolve with clause_i: {'-M', 'N', '-L'} and clause_j: {'-M', 'N', 'L'}
resolvents:  set()
--------------------
Perform resolve with clause_i: {'M', '-N', '-L'} and clause_j: {'-M', 'N', 'L'}
resolvents:  {frozenset({'-L', 'N', '-N', 'L'})}
----------

In [4]:
main("input3.txt", "output3.txt")

Perform read_input input3.txt
alpha:  X
clauses:  [['Y', 'Z', '-X'], ['W', '-Y', '-Z'], ['-W', 'X', 'Y'], ['X', 'Y', 'Z'], ['-X', '-W']]
--------------------
Perform pl_resolution with alpha: X and kb_clauses: [['-W', '-X'], ['-W', 'X', 'Y'], ['-X'], ['-X', 'Y', 'Z'], ['-Y', '-Z', 'W'], ['X', 'Y', 'Z']]
--------------------
Perform resolve with clause_i: {'-X', '-W'} and clause_j: {'X', 'Y', '-W'}
resolvents:  set()
--------------------
Perform resolve with clause_i: {'-X', '-W'} and clause_j: {'-X'}
resolvents:  set()
--------------------
Perform resolve with clause_i: {'-X', '-W'} and clause_j: {'-X', 'Y', 'Z'}
resolvents:  set()
--------------------
Perform resolve with clause_i: {'-X', '-W'} and clause_j: {'-Y', '-Z', 'W'}
resolvents:  set()
--------------------
Perform resolve with clause_i: {'-X', '-W'} and clause_j: {'X', 'Y', 'Z'}
resolvents:  set()
--------------------
Perform resolve with clause_i: {'X', 'Y', '-W'} and clause_j: {'-X'}
resolvents:  {frozenset({'Y', '-W'})}
--

In [5]:
main("input4.txt", "output4.txt")


Perform read_input input4.txt
alpha:  M OR -N OR O
clauses:  [['M', '-N', 'O'], ['O', '-M'], ['-O', '-M'], ['N', 'O', '-M']]
--------------------
Perform pl_resolution with alpha: M OR -N OR O and kb_clauses: [['-M', '-O'], ['-M', 'N', 'O'], ['-M', 'O'], ['-M OR -N OR O'], ['-N', 'M', 'O']]
--------------------
Perform resolve with clause_i: {'-M', '-O'} and clause_j: {'-M', 'N', 'O'}
resolvents:  set()
--------------------
Perform resolve with clause_i: {'-M', '-O'} and clause_j: {'-M', 'O'}
resolvents:  set()
--------------------
Perform resolve with clause_i: {'-M', '-O'} and clause_j: {'-M OR -N OR O'}
resolvents:  set()
--------------------
Perform resolve with clause_i: {'-M', '-O'} and clause_j: {'M', 'O', '-N'}
resolvents:  set()
--------------------
Perform resolve with clause_i: {'-M', 'N', 'O'} and clause_j: {'-M', 'O'}
resolvents:  set()
--------------------
Perform resolve with clause_i: {'-M', 'N', 'O'} and clause_j: {'-M OR -N OR O'}
resolvents:  set()
------------------

In [10]:
main("input5.txt", "output5.txt")

Perform read_input input5.txt
alpha:  P OR Q
clauses:  [['-P', 'Q', '-R'], ['-Q', 'R', '-P'], ['-P', '-Q', 'R']]
--------------------
Perform pl_resolution with alpha: P OR Q and kb_clauses: [['-P', '-Q', 'R'], ['-P', '-Q', 'R'], ['-P', '-R', 'Q'], ['-P OR Q']]
--------------------
Perform resolve with clause_i: {'-Q', 'R', '-P'} and clause_j: {'-Q', 'R', '-P'}
resolvents:  set()
--------------------
Perform resolve with clause_i: {'-Q', 'R', '-P'} and clause_j: {'-R', '-P', 'Q'}
resolvents:  {frozenset({'-Q', '-P', 'Q'})}
--------------------
Perform resolve with clause_i: {'-Q', 'R', '-P'} and clause_j: {'-P OR Q'}
resolvents:  set()
--------------------
Perform resolve with clause_i: {'-Q', 'R', '-P'} and clause_j: {'-R', '-P', 'Q'}
resolvents:  {frozenset({'-Q', '-P', 'Q'})}
--------------------
Perform resolve with clause_i: {'-Q', 'R', '-P'} and clause_j: {'-P OR Q'}
resolvents:  set()
--------------------
Perform resolve with clause_i: {'-R', '-P', 'Q'} and clause_j: {'-P OR Q'}